**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [2]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

FileNotFoundError: [Errno 2] No such file or directory: 'test_images/solidWhiteRight.jpg'

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [3]:
import math
#some global var
global left_line_x1_points
global left_line_x2_points 
global left_line_y1_points 
global left_line_y2_points 
global right_line_x1_points 
global right_line_x2_points 
global right_line_y1_points 
global right_line_y2_points 

left_line_x1_points, left_line_x2_points, left_line_y1_points, left_line_y2_points = [], [], [], []

right_line_x1_points, right_line_x2_points, right_line_y1_points, right_line_y2_points = [], [], [], []

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
   
    mask = np.zeros_like(img)   
    
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
   
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
    
def m_b_finder(lines):
    
    left_lane_lines = []
    left_weights = []
    
    right_lane_lines = []
    right_weights = []
    
    # based on all the bunch of points lines that I get from HT
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            #What if slope is zero, my program already crashed trying to convert a NaN to Int
            if x1==x2:
                    continue
            m_slope = (y2-y1)/(x2-x1) #high school days: definition of line slope
            b_intercept = y1- m_slope*x1
            l = np.sqrt((y2-y1)**2+(x2-x1)**2)
            
            #here is where we classify, remember the coordinates are upside down in an image
            # slope < 0 is the left
            if m_slope < 0:
                right_lane_lines.append((m_slope, b_intercept))
                right_weights.append(l)
            else:
                left_lane_lines.append((m_slope, b_intercept))
                left_weights.append(l)
    #------ At this point I have in two arrays the parameters that describe a line -------#   
    
    left_lane_lines  = np.dot(left_weights,  left_lane_lines) /np.sum(left_weights)  if len(left_weights) >0 else None
    right_lane_lines = np.dot(right_weights, right_lane_lines)/np.sum(right_weights) if len(right_weights)>0 else None
    
    return left_lane_lines, right_lane_lines
                
    
    '''I need to plug the parameters in the image given some coordinates, where:
    
        y1 = coordinate of one point in the bottom of the image 
        y2 = end of the region of interest, '''
   
    #create the points to plug into the equation with the m's and b's
def points_for_line(y1, y2, line):
  
    a_slope, an_intercept = line
    x1 = int((y1-an_intercept)/(a_slope))
    x2 = int((y2-an_intercept)/(a_slope))
    y1 = int(y1)
    y2 = int(y2)
        
        
    return ((x1, y1),(x2, y2))
    
def line_creator(img, lines):
    
    left_lane_lines_params, right_lane_lines_params = m_b_finder(lines)

    # at this point I have an array with tuples containing slope and intercept
        
    y1_point_line = img.shape[0]
    y2_point_line = y1_point_line*0.6
    
    left_line = points_for_line(y1_point_line, y2_point_line, left_lane_lines_params)
    
    right_line= points_for_line(y1_point_line, y2_point_line, right_lane_lines_params)
    
    return left_line, right_line

def actual_drawing(img, lines, color = [255, 0, 0], thickness = 10):
    #print('lines getting into line creator \n', lines)
    left_line, right_line = line_creator(img, lines) 
    left_line = list(left_line)
    right_line = list(right_line)
   
    
    for line in left_line: # i.e ((846, 540), (591, 378))
        pt1 = left_line[0]
        pt2 = left_line[1]
        pt_x1 = left_line[0][0]
        left_line_x1_points.append(pt_x1)
        pt_x2 = left_line[1][0]
        left_line_x2_points.append(pt_x2)
        pt_y1 = left_line[0][1]
        left_line_y1_points.append(pt_y1)
        pt_y2 = left_line[1][1]
        left_line_y2_points.append(pt_y2)
        
        average_x1_point_left_lane = int(np.mean(left_line_x1_points, axis = 0))
        average_y1_point_left_lane = int(np.mean(left_line_y1_points, axis = 0))
        average_x2_point_left_lane = int(np.mean(left_line_x2_points, axis = 0))
        average_y2_point_left_lane = int(np.mean(left_line_y2_points, axis = 0))
        
        if average_x1_point_left_lane < (left_line[0][0]) and average_x2_point_left_lane < left_line[1][0]:
            cv2.line(img, pt1, pt2, color, thickness)
        else:
            cv2.line(img, (average_x1_point_left_lane, average_y1_point_left_lane), (average_x2_point_left_lane, average_y2_point_left_lane), color, thickness)    
            
            
    for line in right_line:
        
        pt_r1 = right_line[0]
        pt_r2 = right_line[1]
        pt_x1_r = right_line[0][0]
        right_line_x1_points.append(pt_x1_r)
        pt_x2_r = right_line[1][0]
        right_line_x2_points.append(pt_x2_r)
        pt_y1_r = right_line[0][1]
        right_line_y1_points.append(pt_y1_r)
        pt_y2_r = right_line[1][1]
        right_line_y2_points.append(pt_y2_r)
        
        average_x1_point_right_lane = int(np.mean(right_line_x1_points, axis = 0))
        average_y1_point_right_lane = int(np.mean(right_line_y1_points, axis = 0))
        average_x2_point_right_lane = int(np.mean(right_line_x2_points, axis = 0))
        average_y2_point_right_lane = int(np.mean(right_line_y2_points, axis = 0))
        
        if average_x1_point_right_lane < right_line[0][0] and average_x2_point_right_lane < right_line[1][0]:
            cv2.line(img, pt_r1, pt_r2, color, thickness)
        else:
            cv2.line(img, (average_x1_point_right_lane, average_y1_point_right_lane), (average_x2_point_right_lane, average_y2_point_right_lane), color, thickness)
                 


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #draw_lines(line_img, lines)
    actual_drawing(line_img, lines)
    return line_img


def weighted_img(img, initial_img, α=0.8, β=0.95, γ=0.):
 
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [4]:
import os
os.listdir("test_images/")

['solidWhiteCurve.jpg',
 'solidYellowCurve.jpg',
 'solidYellowLeft.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidYellowCurve2.jpg',
 'solidWhiteRight.jpg']

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [5]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

In [6]:
def lane_finding_pipeline(image):
    # Parameters neeeded
    
    ks = 15
    low_threshold = 50
    high_threshold = 150
    vertices = np.array([[(40,image.shape[0]), (470, 310),(540,320), (image.shape[1],image.shape[0])]], dtype=np.int32)

    
    #HT Parameters
    rho = 1
    theta = np.pi/180
    threshold = 20
    min_line_len = 5
    max_line_gap = 2
    
    the_image = image
    the_image_gray = grayscale(the_image)
    the_image_blurred = gaussian_blur(the_image_gray, ks)
    the_image_cannied = canny(the_image_blurred, low_threshold, high_threshold)
    the_image_roi = region_of_interest(the_image_cannied, vertices)
    
    the_image_ht_lines = hough_lines(the_image_roi, rho, theta, threshold, min_line_len, max_line_gap)
    
    the_image_weighted = weighted_img(the_image_ht_lines, image)
    
    #plt.imshow(the_image_weighted, cmap = 'gray')
    
    return the_image_weighted

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [7]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [8]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result = lane_finding_pipeline(image)
    return result

Let's try the one with the solid white lane on the right first ...

In [9]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:12<00:00, 15.11it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.34 s, sys: 260 ms, total: 2.6 s
Wall time: 14.1 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [11]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:42<00:00, 15.21it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 8.18 s, sys: 677 ms, total: 8.85 s
Wall time: 44.5 s


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [39]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:32<00:00,  7.44it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 6.25 s, sys: 619 ms, total: 6.87 s
Wall time: 36.7 s


In [40]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))